# Basic Semantic Search
Language models give computers the ability to search by meaning and go beyond searching by matching keywords. This capability is called semantic search.

![Searching an archive using sentence embeddings](https://github.com/cohere-ai/notebooks/raw/main/notebooks/images/basic-semantic-search-overview.png?3)

In this notebook, we'll build a simple semantic search engine. The applications of semantic search go beyond building a web search engine. They can empower a private search engine for internal documents or records. It can also be used to power features like StackOverflow's "similar questions" feature.

1. Get the archive of questions
2. [Embed](https://docs.cohere.ai/embed-reference/) the archive
3. Search using an index and nearest neighbor search
4. Visualize the archive based on the embeddings

In [ ]:
# Install Cohere for embeddings, Umap to reduce embeddings to 2 dimensions,
# Altair for visualization, Annoy for approximate nearest neighbor search
!pip install cohere umap-learn altair annoy datasets tqdm

And if you're running an older version of the SDK, you might need to upgrade it like so:

In [ ]:
#!pip install --upgrade cohere

Get your Cohere API key by [signing up here](https://os.cohere.ai/register). Paste it in the cell below.

## 1. Getting Set up

In [2]:
#@title Import libraries (Run this cell to execute required code) {display-mode: "form"}

import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

You'll need your API key for this next cell. [Sign up to Cohere](https://os.cohere.ai/) and get one if you haven't yet.

## 2. Get The Archive of Questions
We'll use the [trec](https://www.tensorflow.org/datasets/catalog/trec) dataset which is made up of questions and their categories.

In [3]:
# Get dataset
dataset = load_dataset("trec", split="train")

# Import into a pandas dataframe, take only the first 1000 rows
df = pd.DataFrame(dataset)[:1000]

# Preview the data to ensure it has loaded correctly
df.head(10)

,text,coarse_label,fine_label
0,How did serfdom develop in and then leave Russia ?,2,26
1,What films featured the character Popeye Doyle ?,1,5
2,How can I find a list of celebrities ' real names ?,2,26
3,What fowl grabs the spotlight after the Chinese Year of the Monkey ?,1,2
4,What is the full form of .com ?,0,1
5,What contemptible scoundrel stole the cork from my lunch ?,3,29
6,What team did baseball 's St. Louis Browns become ?,3,28
7,What is the oldest profession ?,3,30
8,What are liver enzymes ?,2,24
9,Name the scar-faced bounty hunter of The Old West .,3,29


## 2. Embed the archive
The next step is to embed the text of the questions.

![embedding archive texts](https://github.com/cohere-ai/notebooks/raw/main/notebooks/images/semantic-search-embed-text-archive.png)

To get a thousand embeddings of this length should take about fifteen seconds.

In [4]:
import chromadb
chroma_client = chromadb.Client()

In [10]:
# Get the embeddings
from chromadb.utils import embedding_functions

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

val = sentence_transformer_ef(["foo"])
val

[[0.016233382746577263,
  -0.007662366144359112,
  0.018606502562761307,
  0.031968992203474045,
  -0.031003780663013458,
  0.008778083138167858,
  0.15945537388324738,
  -0.00952159333974123,
  0.020200274884700775,
  -0.04545801505446434,
  0.013985875062644482,
  -0.0176750048995018,
  -0.036169689148664474,
  -0.02194339781999588,
  0.021387742832303047,
  0.06459280848503113,
  -0.036595419049263,
  -0.012133555486798286,
  -0.04366632550954819,
  -0.03515004366636276,
  -0.03262994438409805,
  0.07834126800298691,
  -0.021041665226221085,
  0.03372766450047493,
  -0.02415788173675537,
  -0.01076717209070921,
  -0.042864784598350525,
  0.013539677485823631,
  0.05039738863706589,
  -0.09195670485496521,
  0.03549463674426079,
  0.18029740452766418,
  0.01576359011232853,
  -0.04949156194925308,
  -0.003976411651819944,
  0.0003210735449101776,
  0.021849650889635086,
  0.035368338227272034,
  0.0418541319668293,
  0.04899367317557335,
  -0.026651354506611824,
  -0.0565087869763374

In [12]:
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=sentence_transformer_ef, metadata={"hnsw:space": "cosine"})


In [23]:
embedded_texts = sentence_transformer_ef(df['text'])
embedded_texts = np.array(embedded_texts)
embedded_texts.shape

(1000, 384)

## 3. Search using an index and nearest neighbor search
![Building the search index from the embeddings](https://github.com/cohere-ai/notebooks/raw/main/notebooks/images/semantic-search-index.png)
Let's now use [Annoy](https://github.com/spotify/annoy) to build an index that stores the embeddings in a way that is optimized for fast search. This approach scales well to a large number of texts (other options include [Faiss](https://github.com/facebookresearch/faiss), [ScaNN](https://github.com/google-research/google-research/tree/master/scann), and [PyNNDescent](https://github.com/lmcinnes/pynndescent)).

After building the index, we can use it to retrieve the nearest neighbors either of existing questions (section 3.1), or of new questions that we embed (section 3.2).

In [ ]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

True

### 3.1. Find the neighbors of an example from the dataset
If we're only interested in measuring the distance between the questions in the dataset (no outside queries), a simple way is to calculate the distance between every pair of embeddings we have.

In [ ]:
# Choose an example (we'll retrieve others similar to it)
example_id = 92

# Retrieve nearest neighbors
similar_item_ids = search_index.get_nns_by_item(example_id,10,
                                                include_distances=True)
# Format and print the text and distances
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                             'distance': similar_item_ids[1]}).drop(example_id)

print(f"Question:'{df.iloc[example_id]['text']}'\nNearest neighbors:")
results

Question:'What are bear and bull markets ?'
Nearest neighbors:


,texts,distance
614,What animals do you find in the stock market ?,0.904278
137,What are equity securities ?,0.992819
513,What do economists do ?,1.066583
307,What does NASDAQ stand for ?,1.080738
363,What does it mean `` Rupee Depreciates '' ?,1.086724
932,Why did the world enter a global depression in 1929 ?,1.099370
547,Where can stocks be traded on-line ?,1.105368
922,What is the difference between a median and a mean ?,1.141870
601,What is `` the bear of beers '' ?,1.154140


### 3.2. Find the neighbors of a user query
We're not limited to searching using existing items. If we get a query, we can embed it and find its nearest neighbors from the dataset.

In [ ]:
query = "What is the tallest mountain in the world?"
input_type_query = "search_query"

# Get the query's embedding
query_embed = co.embed(texts=[query],
                  model=model_name,
                  input_type=input_type_query).embeddings

# Retrieve the nearest neighbors
similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                include_distances=True)
# Format the results
query_results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                             'distance': similar_item_ids[1]})


print(f"Query:'{query}'\nNearest neighbors:")
print(query_results) # NOTE: Your results might look slightly different to ours.

Query:'What is the tallest mountain in the world?'
Nearest neighbors:


,texts,distance
236,What is the name of the tallest mountain in the world ?,0.447309
670,What is the highest mountain in the world ?,0.552254
412,What was the highest mountain on earth before Mount Everest was discovered ?,0.801252
907,What mountain range is traversed by the highest railroad in the world ?,0.929516
435,What is the highest peak in Africa ?,0.930806
109,Where is the highest point in Japan ?,0.977315
901,What 's the longest river in the world ?,1.064209
114,What is the largest snake in the world ?,1.076390
962,What 's the second-largest island in the world ?,1.088034
27,What is the highest waterfall in the United States ?,1.091145


## 4. Visualizing the archive
Finally, let's plot out all the questions onto a 2D chart so you're able to visualize the semantic similarities of this dataset!

In [24]:
#@title Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=20)
umap_embeds = reducer.fit_transform(embedded_texts)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'text': df['text']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text']
).properties(
    width=700,
    height=400
)
chart.interactive()

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


alt.Chart(...)

Hover over the points to read the text. Do you see some of the patterns in clustered points? Similar questions, or questions asking about similar topics?

This concludes this introductory guide to semantic search using sentence embeddings. As you continue the path of building a search product additional considerations arise (like dealing with long texts, or finetuning to better improve the embeddings for a specific use case).


We can’t wait to see what you start building! Share your projects or find support on [Discord](https://discord.com/invite/co-mmunity).
